В задании вам понадобится собрать генеративную модлель для языка и спользованием механизма внимания.

In [ ]:
!pip install --quiet sentencepiece datasets transformers

In [ ]:
import random
import torch 
import numpy as np
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [ ]:
# Добавте код для подготовки данных



# Слой внимания (2 балла)



Ниже вам нужно реализовать слой для `MultiheadAttention`.



![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/transformer-attention.png?raw=1)

Механизм внимания состоит из запросов (*queries*), ключей (*keys*) и значений (*values*) - где запрос используется с ключом для получения вектора внимания (обычно результат операции *softmax* и имеет все значения от 0 до 1, сумма которых равна 1), который затем используется для получения взвешенной суммы значений.

Transformer использует *scaled dot-product attention*, где запрос и ключ объединяются путем скалярного произведения между ними, затем применения операции softmax и масштабирования на $d_k$ перед, наконец, умножением на значение. $d_k$ — это *размер головы*, `head_dim`, 
$$ \text{Attention}(Q, K, V) = \text{Softmax} \big( \frac{QK^T}{\sqrt{d_k}} \big)V $$ 

Это похоже на стандартное *dot product attention*, но масштабируется с помощью $d_k$, который используется для предотвращения увеличения результатов скалярных произведений, что приводит к тому, что градиенты становятся слишком маленькими.

Однако *dot product attention* не просто применяется к запросам, ключам и значениям. Вместо того, чтобы делать одно применения внимания, запросы, ключи и значения имеют разделяются $h$ *голов*, и внимание вычисляется для всех головок параллельно. Это означает, что вместо того, чтобы обращать внимание на одну часть последовательности, мы обращаем внимание на $h$. Затем мы обратно объединяем головы в их вектор размерность «hid_dim», таким образом, каждый «hid_dim» потенциально обращает внимание на $h$ разных областей.

$$ \text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1,...,\text{head}_h)W^O $$

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V) $$

$W^O$ линейный слой который применяется в конце слоя, `fc_o`. $W^Q, W^K, W^V$d в модели представлены линейнми слоями `fc_query`, `fc_key` and `fc_value`.

In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, hid_dim, num_heads, attn_dropout=0.1):
        super().__init__()

        self.num_heads = num_heads
        
        assert hid_dim % num_heads == 0, "invalid heads and embedding dimension configuration"
        
        self.fc_key = nn.Linear(hid_dim, hid_dim)
        self.fc_value = nn.Linear(hid_dim, hid_dim)
        self.fc_query = nn.Linear(hid_dim, hid_dim)
        self.attn_dropout = nn.Dropout(attn_dropout)

        self.head_dim = hid_dim // num_heads

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    
    #(batch_size, seq_len, embed_dim)
    def forward(self, q, k, v, mask):
        batch_size = q.size(0)
        seq_len = q.size(1)
        # q.shape, k.shape, v.shape == (batch_size, seq_len, embed_dim)
        k_t = self.fc_key(k).reshape(batch_size, seq_len, self.num_heads, self.head_dim).permute(0, 2, 3, 1)
        # k_t.shape == (batch_size, num_heads, head_dim, seq_len)

        v = self.fc_value(v).reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        # v.shape == (batch_size, num_heads, seq_len, head_dim)

        q = self.fc_query(q).reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        # q.shape == (batch_size, num_heads, seq_len, head_dim)
        
        
        attn = ... # TODO: Your code here
        # attn.shape == (batch_size, num_heads, seq_len, seq_len)


        attn = self.attn_dropout(attn)

        y = torch.matmul(attn, v)
        # y.shape == (batch_size, num_heads, seq_len, head_dim)
        y = y.transpose(1, 2)
        # y.shape == (batch_size, seq_len, num_heads, head_dim)
        y = y.reshape(batch_size, seq_len, -1)
        # y.shape == (batch_size, seq_len, embed_dim)
        y = self.fc_o(y)
        
        return attn, y

Для удобства будем хранить конфигурацию для модели в отдельном классе. 

In [ ]:
class GPTConfig:
    attn_dropout = 0.1
    embed_dropout = 0.1
    ff_dropout = 0.1
    num_heads = 4
    num_blocks = 2
    embed_dim = 512
    
    def __init__(
        self, vocab_size, max_len, **kwargs
    ):
        self.vocab_size = vocab_size
        self.max_len = max_len
        for key, value in kwargs.items():
            setattr(self, key, value)    

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.act = nn.GELU()
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [batch size, seq len, hid dim]  
        x = self.dropout(self.act(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.embed_dim
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.attention = MultiheadAttention(embed_dim, config.num_heads, config.attn_dropout)

        self.ff = PositionwiseFeedforwardLayer(embed_dim, embed_dim * 4, config.ff_dropout) 
    
    def forward(self, x, mask):
        x = self.ln1(x)
        
        attn, dx = self.attention(x, x, x, mask)

        x = x + dx
        
        x = x + self.ff(self.ln2(x))
        return attn, x

# Обучните модель для генерации тескта (4 балла)

Мы не хотим чтобы наша модель заглядывала в будущее. Для этого мы создаем маску по которой текущий токен может только смотреть на себя и на предыдущие. Для этого нужна маска у которой элементы над главной диагональю нулевые. Для этого можно использовать `torch.tril`. 
Пример масти для последовательность длины 5:

$$\begin{matrix}
1 & 0 & 0 & 0 & 0\\
1 & 1 & 0 & 0 & 0\\
1 & 1 & 1 & 0 & 0\\
1 & 1 & 1 & 1 & 0\\
1 & 1 & 1 & 1 & 1\\
\end{matrix}$$

In [ ]:
def make_mask(seq):
    ... # TODO: Your code here
    return mask

Дополните код для модели

In [ ]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.embed_dim
        self.max_len = config.max_len
        self.tok_embed = 
        self.pos_embed = nn.Parameter(
            torch.zeros(1, config.max_len, embed_dim)
        )

        self.blocks = nn.ModuleList(
            [Block(config) for _ in range(config.num_blocks)]
        )
        self.ln = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, config.vocab_size)
    
    def forward(self, token_indexes):
        # batch_size = x.size(0)
        seq_len = token_indexes.size(1)
        assert seq_len <= self.max_len, "sequence longer than model capacity"
        
        tok_embedding = ... # TODO: Your code here

        # tok_embedding.shape == (batch_size, seq_len, embed_dim)

        pos_embedding = ... # TODO: Your code here

        # pos_embedding.shape == (1, seq_len, embed_dim)
        
        x = self.dropout(tok_embedding + pos_embedding)

        seq_len = x.size(1)

        mask = make_mask(token_indexes)

        # Примените все блоки последовательно
        # TODO: Your code here

        ...        

        x = self.ln(x)
        x = self.fc(x)
        # x.shape == (batch_size, seq_len, vocab_size)
        return attn_list, x

In [ ]:
tokenizer.vocab_size

8000

In [ ]:
vocab_size = tokenizer.vocab_size

config = GPTConfig(vocab_size, max_seq_len)
model = GPT(config).to(device)

In [ ]:
batch = next(iter(train_dataloader))
model(batch[:, :-1].to(device))[1].shape

torch.Size([16, 256, 8000])

In [ ]:
model = model.to(device)

In [ ]:
learning_rate = 0.0005

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index = pad_token_idx)

In [ ]:
# Реализуйте обучение, так же как в предущей тетрадке
def train_epoch(model, callback):
    ...


In [ ]:
def eval_model(model):
    ...

Подберити варианты количетва блоков и количетсва голов при которых модель дает хороший результат. 
Получите `loss < 4.5`

In [ ]:
def callback(train_loss):
    eval_loss = eval_model(model)
    model.train()
    print(f'Epoch: {epoch+1:02} | train_loss = {train_loss:.5f}, eval_loss = {eval_loss:.5f}')

for epoch in trange(1):
    train_loss = train_epoch(model, callback)

In [ ]:
tokenizer.encode("я помню чудное мгновенье")

[2, 156, 2769, 2723, 539, 2646, 1881, 3]

In [ ]:
def continues_sentence(sentence, model, max_len = 30):
    # Возмите код из прошлого задания

In [ ]:
continues_sentence("Я помню чудное мгновенье", model)

'я помню чудное мгновенье с ужасным чувством.. он мне думает о каком - то особенном положении : он еще не бывал и снова не было. если начинают головы'

In [ ]:
continues_sentence("Мой дядя самых честных правил,", model)

'мои дядя самых честных правил, законы мои жорея : прежде всякии сказал :, что лютера, без вас же разумнымство останется древним народам'

In [ ]:
continues_sentence("Четыре года потратил Деонардо на", model)

'четыре года потратил деонардо на другои вопрос с потаевыми классами и, чувствуя от всех на волеи и очень трудно просить рент либеральную типу'

In [ ]:
continues_sentence("если крикнет рать святая", model)

'если крикнет рать святая жида. собрав фисию владимир, как будто он по самыи заслужен не даст, что, что другого оттенка проска'

In [ ]:
continues_sentence("Он пересел на свой стул, придвинул к себе суп, говядину и стал ", model)

'он пересел на свои стул, придвинул к себе суп, говядину и стал за ним спокоиности, и настоичивал на нее, всю чаику : " счастье мне удалось это заключаться в'